In [48]:
%%html
<style>
table {margin-left: 0 !important;}
</style>

In [88]:
from IPython.display import display, HTML

import re
import io
import pandas as pd
from pandas.util.testing import assert_frame_equal

class TableData:
    def __init__(self, table):
        self.df = self.as_df(table)

    def as_df(self, table):
        # Remove trailing comments
        table_to_parse = re.compile(r'(#.*$)', flags = re.MULTILINE).sub('', table)
        
        # Remove beginning and terminal pipe on each row
        table_to_parse = re.compile(r'(^\s*\|\s*|\s*\|\s*$)', flags = re.MULTILINE).sub('', table_to_parse)
        
        # Remove whitespace surrouding pipes
        table_to_parse = re.compile(r'\s*\|\s*').sub('|', table_to_parse)
        
        # Remove header separator
        table_to_parse = table_to_parse.split('\n')
        table_to_parse.pop(1)
        table_to_parse = '\n'.join(table_to_parse)

        parsed_df = pd.read_csv(io.StringIO(table_to_parse), sep="|")
        return TableData.stringify_df(parsed_df)

    @classmethod
    def stringify_df(cls, df):
        for col in df.columns:
            df[col] = df[col].apply(lambda x: str(x))
        return df

    @classmethod
    def compare_stringified_df(cls, df1, df2):
        df1 = cls.stringify_df(df1.copy())
        df2 = cls.stringify_df(df2.copy())
        return df1.equals(df2)


import nbformat

def get_markdown_table(scenario, table_name):
    fp = open('SomeFeatures.ipynb')
    nb = nbformat.read(fp, 4)

    table_cells = [cell for cell in nb['cells'] if 'table_name' in cell['metadata']]
    table_data = {}
    for cell in table_cells:
        cell_scenario = cell['metadata']['scenario']
        cell_table_name = cell['metadata']['table_name']
        
        if cell_scenario not in table_data:
            table_data[cell_scenario] = {}
        table_data[cell_scenario][cell_table_name] = cell['source']
        
    return TableData(table_data[scenario][table_name]).df

#get_markdown_table('simple_mapper', 'target')

,Student_ID__c,FirstName,Phone
0,1,Buffy,555-555-5555


## Simple mapper

Given the following source:

| id | first_name | phone |
| - | - | - |
| 1  | Buffy | 555-555-5555 |

Then the target should match the example:

| Student_ID__c | FirstName | Phone |
| - | - | - |
| 1  | Buffy | 555-555-5555 |

In [89]:
def simple_map_job(source):
    target_df = pd.DataFrame([])
    target_df['Student_ID__c'] = source['id']
    target_df['FirstName'] = source['first_name']
    target_df['Phone'] = source['phone']
    return target_df


result_df = simple_map_job(get_markdown_table('simple_mapper', 'source'))
assert_frame_equal(result_df, get_markdown_table('simple_mapper', 'target'))

## Simple transform

Given the following source called 'MyFile':

| id | name |
| - | - |
| 1  | Summers,Buffy |

Then the target 'Catalyst Contact' should match the example:

| Student_ID__c | FirstName | LastName |
| - | - | - | - |
| SDH1  | Buffy | Summers |

In [91]:
def simple_transform_job(source):
    target_df = pd.DataFrame([])
    target_df['Student_ID__c'] = source['id'].apply(lambda x: 'SDH{}'.format(x))
    target_df['FirstName'] = source['name'].apply(lambda x: x.split(',')[1])
    target_df['LastName'] = source['name'].apply(lambda x: x.split(',')[0])
    return target_df


result_df = simple_transform_job(get_markdown_table('simple_transform', 'source'))
assert_frame_equal(result_df, get_markdown_table('simple_transform', 'target'))

## Deduper

Given the following source called 'MyFile':

| id | name |
| - | - |
| 1  | Buffy |
| 1  | Faith |

Then the target 'Catalyst Contact' should match the example:

| Student_ID__c | FirstName |
| - | - | - |
| SDH1  | Buffy |

In [102]:
def deduper_job(source):
    target_df = pd.DataFrame([])
    target_df['Student_ID__c'] = source['id'].apply(lambda x: 'SDH{}'.format(x))
    target_df['FirstName'] = source['name']
    target_df = target_df.drop_duplicates('Student_ID__c', keep='first')
    return target_df


result_df = deduper_job(get_markdown_table('deduper', 'source'))
assert_frame_equal(result_df, get_markdown_table('deduper', 'target'))

## Join

Given the following source called 'MyFile':

| id | name |
| - | - |
| 1  | Buffy |

And the following source called 'ClassesFile':

| id | student_id | name |
| - | - | - |
| 1 | 1 | Psychology |
| 2 | 1 | History |
| 3 | 1 | Physiology |

Then the target 'Catalyst Contact' should match the example:

| Class_Record_ID__c | FirstName | ClassName |
| - | - | - |
| SDH1-1  | Buffy | Psychology |
| SDH1-2  | Buffy | History |
| SDH1-3  | Buffy | Physiology |

In [117]:
def join_job(contacts_df, classes_df):
    work_joined_df = contacts_df.merge(classes_df, how='inner', left_on=['id'], right_on=['student_id'], suffixes=('__contact', '__class'))
    
    result_df = pd.DataFrame([])
    result_df['Class_Record_ID__c'] = work_joined_df.apply(lambda row: "SDH{}-{}".format(row['id__contact'], row['id__class']), axis=1)
    result_df['FirstName'] = work_joined_df['name__contact']
    result_df['ClassName'] = work_joined_df['name__class']
    return result_df


result_df = join_job(get_markdown_table('join', 'contacts'), get_markdown_table('join', 'classes'))
assert_frame_equal(result_df, get_markdown_table('join', 'target'))